# DL MP3 - Basma ESSID

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

C:\Users\leade\AppData\Local\Enthought\Canopy\edm\envs\python36\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [80]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [5]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act returns the next action to do based on a random exploration of ```epsilon```. Since ```epsilon``` controls the exploration-exploitation tradeoff, the function act ensures that the agent will explore the action space with a probability of ```epsilon``` and will exploit its discoveries with a probability of ```1-epsilon```.

In the training part of the function act (when ```train``` is True), the agent will, depending on the ```epsilon``` value and the randomly regenerated number in the condition, explore the action space in order to learn policies and increase the chances to maximize the reward, or exploit them. Both exploration and exploitation are needed.

In the testing part of the function act (when ```train``` is False), the function act is using a pure exploitation strategy (as if ```epsilon``` equals 0). After training, we may indeed consider that there is no need for further exploration. 

The difference with the training part is that while training, the need for exploration decreases with iterations (inversely proportional to the need for exploitation) due to the optimization of Q estimates over time.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [2]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [3]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=101 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

__Board Array__: This array represents the board in which the game is played. It encodes the agent's rewards. It is initialized as a ```grid_size²``` grid. Each cell is either a bonus (reward= +0.5) with probability temperature or a malus (reward=-1) with the same probability, otherwize when there is nothing, the reward is 0. Throughout the game, the agent changes its position on the board and goes from cell to cell and then receives each correspondent reward. That reward is attributed only once.

__Position Array__: This array represents the agent's position in the grid environment (Defined as +1), it is initialized at (x,y)=(0,1), and it indicates the possible moves for the agent (precise the boards with -1 value). The agent can see up to 2 position cells around him (thus it can see 25² cells).

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        
        # Initialize the environment with a random state
        state = env.reset()
        
        game_over = False
        
        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                
        
        if e%10 ==0:
            # Save as a mp4
            env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [14]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 8.5/20.0. Average score (-11.5)
Win/lose count 11.0/10.0. Average score (-5.25)
Win/lose count 6.0/9.0. Average score (-4.5)
Win/lose count 9.5/17.0. Average score (-5.25)
Win/lose count 7.0/9.0. Average score (-4.6)
Win/lose count 6.5/8.0. Average score (-4.083333333333333)
Win/lose count 5.5/15.0. Average score (-4.857142857142857)
Win/lose count 11.0/20.0. Average score (-5.375)
Win/lose count 7.0/12.0. Average score (-5.333333333333333)
Win/lose count 8.0/22.0. Average score (-6.2)
Win/lose count 13.5/15.0. Average score (-5.7727272727272725)
Final score: -5.7727272727272725


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Question 5.1 :__
We note the $Q$-function:

\begin{align*}
Q^{\pi}(s,a)&=E_{p^{\pi}}[\sum_{0\leq t}\gamma^tr(s_t,a_t)\vert s_0=s, a_0=a] \\
&= E_{p^{\pi}}[r(s_0,a_0) + \sum_{1\leq t}\gamma^tr(s_t,a_t)\vert s_0=s,a_0=a] \\
&= E_{p^{\pi}}[r(s,a) + \sum_{1\leq t}\gamma^tr(s_t,a_t)\vert s_0=s,a_0=a] \\
&= r(s,a) +  E_{p^{\pi}}[ \gamma \sum_{0\leq t}\gamma^tr(s_{t+1},a_{t+1})\vert s_0=s,a_0=a] \\
&= r(s,a) + \sum_{a'}\sum_{s'}p(s_1=s',a_1=a'|s_0=s,a_0=a)  \gamma E_{p^{\pi}}[  \sum_{0\leq t}\gamma^tr(s_{t+1},a_{t+1})\vert s_0=s,a_0=a,s_1=s',a_1=a'] \\
&= r(s,a) + \sum_{a'}\sum_{s'}p(s_1=s',a_1=a'|s_0=s,a_0=a)  \gamma E_{p^{\pi}}[  \sum_{0\leq t}\gamma^tr(s_{t+1},a_{t+1})\vert s_1=s',a_1=a'] \\
&= r(s,a) + \sum_{a'}\sum_{s'}p(s_1=s',a_1=a'|s_0=s,a_0=a)  \gamma Q^{\pi}(s',a') \\
&= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\ 
\end{align*}


__Question 5.2 :__

Assuming the existence of the optimal policy $\pi^*$,
\begin{align*}
    Q^*(s,a)
    &= \max_{\pi}Q^\pi(s,a) \\
    &=  \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\ 
    &=  r(s,a) +  \max_{\pi} \sum_{a'} \sum_{s'} p(s_1=s', a_1=a' |s_{0}=s,a_{0}=a) \gamma Q^{\pi}(s',a') \\   
    &=  r(s,a) +  \max_{\pi} \sum_{a'} \sum_{s'} p(s_1=s'|s_{0}=s,a_{0}=a) \pi(a' | s') \gamma Q^{\pi}(s',a') \\ 
    &=  r(s,a) +   \sum_{a'} \sum_{s'} p(s_1=s'|s_{0}=s,a_{0}=a) \max_{\pi} \pi(a' | s') \gamma Q^{\pi}(s',a') \\ 
    &=  r(s,a) +   \sum_{a'} \sum_{s'} p(s_1=s'|s_{0}=s,a_{0}=a)  \gamma \max_{a'} Q^{*}(s',a') \\ 
    &= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')] \\
\end{align*}


__Question 5.3 :__

The plausible objective is the loss function which indicates how well our prediction are in comparison with the actual target (We are actually calculating the distance between the prediction and the target). 

We want to decrease this gap between the prediction of our model $Q(s,a;\theta)$ and the target $E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]$ by adjusting the value of ```theta```

So we obtain: 
$$ 
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}
$$



***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [8]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        # If the max_memory is hit, remove the first element
        if len(self.memory)>=self.max_memory:
            del self.memory[0]
        self.memory.append(m)

    def random_access(self):
        # Random access of list
        return self.memory[np.random.choice(range(len(self.memory)))]

***
The pipeline we will use for training is given below:

In [9]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [10]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        predicted_act = self.model.predict(s.reshape(1,5,5,self.n_state))
        return np.argmax(predicted_act)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s, n_s, a, r, game_over = self.memory.random_access()
            input_states[i]=s
            target_q[i] = self.model.predict(s.reshape(1,5,5,self.n_state))[0]
            
            if game_over_:
                ######## FILL IN
                target_q[i][a] = r
            else:
                ######## FILL IN
                target_q[i][a]  = r + self.discount*max(self.model.predict(n_s.reshape(1,5,5,self.n_state))[0])
                
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Dense(32,activation='relu'))
        model.add(Dense(32,activation='relu'))
        model.add(Dense(4,activation='relu'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        model.summary()
        

In [11]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1632      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 132       
Total params: 2,820
Trainable params: 2,820
Non-trainable params: 0
_________________________________________________________________
Epoch 000/101 | Loss 0.0360 | Win/lose count 7.0/12.0 (-5.0)
Epoch 001/101 | Loss 0.0544 | Win/lose count 8.5/1.0 (7.5)
Epoch 002/101 | Loss 0.1997 | Win/lose count 6.0/1.0 (5.0)
Epoch 003/101 | Loss 0.4484 | Win/lose count 4.0/5.0 (-1.0)
Epoch 004/101 | Loss 1.6568 | Win

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(32, (3, 3),activation='relu',input_shape=(5,5,self.n_state)))
        model.add(Conv2D(32, (3, 3),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4, activation='relu'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        model.summary()

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 3, 3, 32)          608       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 32)          9248      
_________________________________________________________________
flatten_2 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 132       
Total params: 9,988
Trainable params: 9,988
Non-trainable params: 0
_________________________________________________________________
Epoch 000/101 | Loss 0.0147 | Win/lose count 2.5/4.0 (-1.5)
Epoch 001/101 | Loss 0.0147 | Win/lose count 3.0/4.0 (-1.0)
Epoch 002/101 | Loss 0.0181 | Win/lose count 2.0/3.0 (-1.0)
Epoch 003/101 | Loss 0.0143 | Win/lose count 2.0/1.0 (1.0)
Epoch 004/101 | Loss 0.0305 | Win

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 3, 32)          608       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 1, 1, 32)          9248      
_________________________________________________________________
flatten_3 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 132       
Total params: 9,988
Trainable params: 9,988
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_6 

In [16]:
HTML(display_videos('cnn_test10.mp4'))

In [17]:
HTML(display_videos('fc_test10.mp4'))

__Comparaison of performances between fully-connected layers (FC) and CNN :__

- CNN clearly outperforms FC model in terms of efficiency. FC model has better results but needs more epochs than CNN to obtain similar training results. When it comes to testing results, the difference of efficiency is even higher. CNN got a positive final score and FC got a negative one. (FC only gets better results when the initial score is high)

__Issues observed :__

- We observe that the rat (agent) usually get stuck in a certain number of visited cells (especially near the corners). In fact, the learning process encourages the agent to stay in visited cells (where reward=0) so that the score does not get worse, the agent does not explore and search for what is behind malus cells.

Solution: 

    -> Penalize the agent when it goes in visited cells while training.
    -> Augment the temperature
    
__The temperature effect on the behavior :__

- The temperature is what controls the probability of having bonuses and maluses, so a higher temperature means more bonus and malus cells. Since there is the operation malus[bonus>0]=0, this ensures that there is more bonus cells than malus cells.
- The ratio bonus/malus cells is higher than value and go higher with a higher value of temperature.
- The result is that the agent has more positive rewards to collect and so that the final reward is better.


In [12]:
for temp in [0.5, 0.7, 0.9]:
    print("------------ Temperature = {} --------------".format(temp))
    env = Environment(grid_size=size, max_time=T,temperature=temp) 
    agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')
    
    agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
    print('Test of the CNN')
    test(agent_cnn,env,epochs_test,prefix='cnn_test')
    print('Test of the FC')
    test(agent_fc,env,epochs_test,prefix='fc_test')

------------ Temperature = 0.5 --------------
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 3, 3, 32)          608       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 32)          9248      
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 132       
Total params: 9,988
Trainable params: 9,988
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 50)                0         
____________________________

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [49]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    # Discount for Q learning
    eps_discount=0.99
    epsilon= agent.epsilon
    
    # Same agent training function as before but incorporating epsilon this time
    for e in range(epoch):

        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        agent.set_epsilon(epsilon*eps_discount)
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        

# This class will inherit Environment
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        Environment.__init__(self, grid_size, max_time, temperature)
        #Add the new state that penalizes being on visited cells
        self.malus_position = np.zeros((self.grid_size,self.grid_size))

        def act(self, action, train=False):
        """This function gets the state, reward, and game over state, penalizes
        being on visited cells then returns the new state, reward and decides 
        if the game ends."""
        
        #Use the act function from the Environment class 
        state, reward, game_over = Environment.act(self,action)
        
        #When training, penalize the agent when revisiting a cell. 
        #This is ensured by setting a reward equal to -0.1 to an already visited cell
        if train:
            reward = reward-self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        
        #if train:
        #    reward = reward - self.malus_position[self.x, self.y]
        #self.malus_position[self.x, self.y] = 0.1
        
        #Incorporating the new feature state
        visited_positions = self.malus_position.reshape(self.grid_size, self.grid_size,1)[self.x-2:self.x+3,self.y-2:self.y+3,:]
        state = np.concatenate((visited_positions, state),axis=2)

        return state, reward, game_over
    
    def reset(self):
        """This function resets the game and returns the initial state"""
        state = Environment.reset(self) 
        
        #We intialize self.malus_position 
        #We can fix the issue of the agent being stuck near the corners by attributing 
        #the value 1 to border cells in self.malus_position
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        self.malus_position[-2:,:]=1
        self.malus_position[:,-2:]=1
        self.malus_position[:2,:]=1
        self.malus_position[:,:2]=1
        self.malus_position[self.x, self.y] = 0.1
        
        visited_positions = self.malus_position.reshape(self.grid_size, self.grid_size,1)[self.x-2:self.x+3,self.y-2:self.y+3,:]
        state = np.concatenate((visited_positions, state),axis=2)

        return state

In [50]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore100.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 3, 3, 32)          896       
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 1, 1, 32)          9248      
_________________________________________________________________
flatten_22 (Flatten)         (None, 32)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 4)                 132       
Total params: 10,276
Trainable params: 10,276
Non-trainable params: 0
_________________________________________________________________
Epoch 000/101 | Loss 0.0121 | Win/lose count 3.0/21.50000000000002 (-18.50000000000002)
Epoch 001/101 | Loss 0.0205 | Win/lose count 2.0/21.000000000000032 (-19.000000000000032)
Epoch 002/101 | Loss 0.0024 | Win/lose count 4.0/22.400000000000034 (-18.400000000000034)
Ep

In [ ]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***